In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping

import os

In [2]:
import splitfolders

input_dir = r'C:\DATA\simpsons_dataset'
output_dir = r'C:\DATA\data'

splitfolders.ratio(input_dir, output=output_dir, seed=1337, ratio=(.8, .1, .1))

ModuleNotFoundError: No module named 'splitfolders'

In [2]:
train_dir = r'C:\DATA\data\train'
val_dir = r'C:\DATA\data\val'
test_dir = r'C:\DATA\data\test'

image_size = (64, 64, 3)
epochs = 50
batch_size = 

labels = os.listdir(train_dir)

In [17]:
labels

['abraham_grampa_simpson',
 'agnes_skinner',
 'apu_nahasapeemapetilon',
 'barney_gumble',
 'bart_simpson',
 'carl_carlson',
 'charles_montgomery_burns',
 'chief_wiggum',
 'cletus_spuckler',
 'comic_book_guy',
 'disco_stu',
 'edna_krabappel',
 'fat_tony',
 'gil',
 'groundskeeper_willie',
 'homer_simpson',
 'kent_brockman',
 'krusty_the_clown',
 'lenny_leonard',
 'lionel_hutz',
 'lisa_simpson',
 'maggie_simpson',
 'marge_simpson',
 'martin_prince',
 'mayor_quimby',
 'milhouse_van_houten',
 'miss_hoover',
 'moe_szyslak',
 'ned_flanders',
 'nelson_muntz',
 'otto_mann',
 'patty_bouvier',
 'principal_skinner',
 'professor_john_frink',
 'rainier_wolfcastle',
 'ralph_wiggum',
 'selma_bouvier',
 'sideshow_bob',
 'sideshow_mel',
 'simpsons_dataset',
 'snake_jailbird',
 'troy_mcclure',
 'waylon_smithers']

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = image_size[:2],
                                                 batch_size=batch_size,
                                                 class_mode = 'sparse',
                                                 color_mode = 'rgb')

val_set = test_datagen.flow_from_directory(val_dir,
                                           target_size = image_size[:2],
                                           batch_size=batch_size,
                                           class_mode = 'sparse',
                                           color_mode = 'rgb')

Found 16727 images belonging to 43 classes.
Found 2073 images belonging to 43 classes.


In [6]:
model = Sequential()
model.add(Input(shape=image_size))
model.add(Conv2D(32, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation = 'softmax'))

In [ ]:
input_layer = Input(shape=image_size)

x1 = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(input_layer)
x2 = Conv2D(32, (3, 3), activation = 'relu')(x1)
x3 = MaxPooling2D(pool_size=(2, 2))(x2)
x4 = Dropout(0.2)(x3)
x5 = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x4)
x6 = MaxPooling2D(pool_size=(2, 2))(x5)
x7 = Dropout(0.2)(x6)
x8 = Conv2D(256, (3, 3), padding = 'same', activation = 'relu')(x7)
x9 = MaxPooling2D(pool_size=(2, 2))(x8)
x10 = Dropout(0.2)(x9)
x11 = Flatten()(x10)
x12 = Dense(1024, activation = 'relu')(x11)
x13 = Dropout(0.5)(x12)
x14 = Dense(len(labels), activation = 'softmax')(x13)

In [7]:
stopCB = EarlyStopping(monitor='val_acc', patience=5)

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 31, 31, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                    

In [11]:
stopCB = EarlyStopping(patience=5, monitor='val_acc')

In [12]:
history = model.fit(training_set,
                    epochs=epochs,
                    validation_data = val_set,
                    callbacks = [stopCB])

Epoch 1/50
168/168 [==============================] - 134s 798ms/step - loss: 2.4731 - acc: 0.3095 - val_loss: 1.8109 - val_acc: 0.5094
Epoch 2/50
168/168 [==============================] - 152s 906ms/step - loss: 2.0301 - acc: 0.4399 - val_loss: 1.4644 - val_acc: 0.5914
Epoch 3/50
168/168 [==============================] - 143s 849ms/step - loss: 1.7156 - acc: 0.5223 - val_loss: 1.1291 - val_acc: 0.6855
Epoch 4/50
168/168 [==============================] - 134s 796ms/step - loss: 1.4806 - acc: 0.5874 - val_loss: 1.0284 - val_acc: 0.7106
Epoch 5/50
168/168 [==============================] - 142s 843ms/step - loss: 1.2964 - acc: 0.6379 - val_loss: 0.8225 - val_acc: 0.7646
Epoch 6/50
168/168 [==============================] - 100s 594ms/step - loss: 1.1774 - acc: 0.6724 - val_loss: 0.7150 - val_acc: 0.8013
Epoch 7/50
168/168 [==============================] - 144s 859ms/step - loss: 1.0928 - acc: 0.6895 - val_loss: 0.6681 - val_acc: 0.8114
Epoch 8/50
168/168 [============================

In [16]:
import pandas as pd

pd.DataFrame(history.history)

,loss,acc,val_loss,val_acc
0,2.473080,0.309500,1.810910,0.509407
1,2.030052,0.439888,1.464407,0.591413
2,1.715649,0.522269,1.129146,0.685480
3,1.480553,0.587374,1.028398,0.710564
4,1.296354,0.637891,0.822520,0.764592
5,1.177423,0.672386,0.715000,0.801254
6,1.092765,0.689484,0.668057,0.811384
7,1.026778,0.709870,0.626178,0.827786
8,0.943656,0.735577,0.624540,0.822479
9,0.907322,0.742811,0.593891,0.832127
